In [18]:
import numpy as np
from scipy.integrate import quad
from datetime import datetime as dt
from datetime import timedelta
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import skew

INPUT PARAMETERS DEFINITION

In [19]:
#S0 INITIAL SPOT PRICE
start = dt(2023,1,1)
end = dt.today()

sp500_ts = yf.download('^GSPC', start = start, end=dt.today())
sp500_ts = sp500_ts['Close']

price_today = np.round(sp500_ts.tail(1)[0])

print('S&P500 price:',price_today)

[*********************100%***********************]  1 of 1 completed
S&P500 price: 4104.0


In [20]:
#r RISK FREE RATE
r = 0.0352

In [21]:
#kappa REVERSE MEAN COMPUTED USING AUTOCORRELATION METHOD
sp500_logret = np.log(sp500_ts/sp500_ts.shift(1)).dropna()
autocorr = sp500_logret.autocorr()
reverse_mean = np.round(np.log(-autocorr), 2)
print("Speed of reversion to the mean:",reverse_mean)

Speed of reversion to the mean: -3.24


In [22]:
#sigma VOLATILITY OF VARIANCE
#vix = yf.download("^VIX", start = start, end=dt.today())
#vix = np.round(vix['Close'].tail(1)[0])

vix_hist = yf.download("^VIX", start = start, end=dt.today())
vix_hist = vix_hist['Close']

vix_ret = np.log(vix_hist/vix_hist.shift(1)).dropna()

sigma = np.round(np.std(vix_ret), 2)
print('Volatility of variance:',sigma)

[*********************100%***********************]  1 of 1 completed
Volatility of variance: 0.06


In [23]:
#Rho CORRELATION BETWEEN UNDERLYING AND HIS VARIANCE
corr_coef = np.round(np.corrcoef(sp500_logret.values, vix_ret.values), 2)
print(corr_coef[0,1])

-0.76


HESTON MODEL PARAMETER DEFINITION

In [24]:
S0 = price_today    # initial spot price
V0 = 0.05   # initial variance
kappa = reverse_mean   # speed of reversion
theta = 0.06  # long term variance
#sigma already defined
rho = corr_coef[0,1]    # correlation between underlying and his variance

PUT OPTION PARAMETER DEFINITION

In [25]:
K = 3810     # Strike price
T = 1/12  # Time to maturity 

MONTE-CARLO SIMULATIONS

In [26]:
# Number of Monte-Carlo simulations
N = 1000000
# Monte-Carlo paths for price and varianxce
dt = T/21
days = 21
Z1 = np.random.randn(N, days)  #Estrazioni casuali da una distribuzione N(0,1)
Z2 = rho*Z1 + np.sqrt(1-rho**2)*np.random.randn(N, days)  #
S = np.zeros((N, days+1))
V = np.zeros((N, days+1))
S[:,0] = S0  #SPOT PRICE
V[:,0] = V0  #VARIANCE
for i in range(days):
    dS = r*S[:,i]*dt + np.sqrt(V[:,i])*S[:,i]*np.sqrt(dt)*Z1[:,i] #Delta prezzo
    dV = kappa*(theta-V[:,i])*dt + sigma*np.sqrt(V[:,i])*np.sqrt(dt)*Z2[:,i] #Delta varianza
    S[:,i+1] = S[:,i] + dS  
    V[:,i+1] = V[:,i] + dV  
    V[:,i+1] = np.maximum(V[:,i+1], 0)  #Confronto la var della montecarlo al tempo t con 0, prendo il massimo o 0, non può essere negativa

In [27]:
# PAYOFF PUT OPTION
payoff = np.maximum(K-S[:,-1], 0)

In [29]:
# PROBABILITY OPTION PUT IN THE MONEY
prob = np.sum(payoff > 0)/N
print("Probability that option put will be ITM in one month's time:", prob)

Probability that option put will be ITM in one month's time: 0.120704
